<a href="https://colab.research.google.com/github/swainsubrat/Bookshop-Automation-Sys/blob/master/Pytorch_image_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install kaggle

In [0]:
#for importing files
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"swainsubrat","key":"6d23107792d3454d4ab8b282eb60dc4b"}'}

In [0]:
#for making a kaggle directory
!mkdir -p ~/.kaggle

#changing the path and moving the file to kaggle folder
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [0]:
#data download by api
!kaggle competitions download -c humpback-whale-identification

  0% 0.00/498k [00:00<?, ?B/s]
100% 498k/498k [00:00<00:00, 67.5MB/s]
  0% 0.00/594k [00:00<?, ?B/s]
100% 594k/594k [00:00<00:00, 82.1MB/s]
 99% 1.33G/1.35G [00:11<00:00, 106MB/s] 
100% 1.35G/1.35G [00:11<00:00, 128MB/s]
100% 4.15G/4.16G [00:30<00:00, 105MB/s]
100% 4.16G/4.16G [00:30<00:00, 148MB/s]


In [0]:
ls

kaggle.json   sample_submission.csv  train.csv
sample_data/  test.zip               train.zip


In [0]:
import pandas as pd

df = pd.read_csv("sample_submission.csv")
df.head()

,Image,Id
0,00028a005.jpg,new_whale w_23a388d w_9b5109b w_9c506f6 w_0369a5c
1,000dcf7d8.jpg,new_whale w_23a388d w_9b5109b w_9c506f6 w_0369a5c
2,000e7c7df.jpg,new_whale w_23a388d w_9b5109b w_9c506f6 w_0369a5c
3,0019c34f4.jpg,new_whale w_23a388d w_9b5109b w_9c506f6 w_0369a5c
4,001a4d292.jpg,new_whale w_23a388d w_9b5109b w_9c506f6 w_0369a5c


In [0]:
train_data = pd.read_csv("train.csv")
train_data.head()

,Image,Id
0,0000e88ab.jpg,w_f48451c
1,0001f9222.jpg,w_c3d896a
2,00029d126.jpg,w_20df2c5
3,00050a15a.jpg,new_whale
4,0005c1ef8.jpg,new_whale


In [0]:
#importing necessary libraries
import numpy as np 
import os
import matplotlib.pyplot as plt
import matplotlib.image as mplimg

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

import torch
from torch.utils.data import TensorDataset, DataLoader,Dataset
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torch.optim import lr_scheduler
import time 
import tqdm
from PIL import Image
train_on_gpu = True
from torch.utils.data.sampler import SubsetRandomSampler
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau, CosineAnnealingLR

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

from collections import OrderedDict
import cv2

In [0]:
torch.cuda.is_available()

True

In [0]:
!pip install pretrainedmodels > /dev/null 2>&1

In [0]:
import albumentations
from albumentations import torch as AT
import pretrainedmodels

In [0]:
!unzip train.zip

Archive:  train.zip
  inflating: 1cb7e70be.jpg           
  inflating: 85fbaaeee.jpg           
  inflating: 0df6c0ada.jpg           
  inflating: b45879784.jpg           
  inflating: 667c90122.jpg           
  inflating: d78425146.jpg           
  inflating: 52fb69ef4.jpg           
  inflating: e6b70421a.jpg           
  inflating: 57c35256c.jpg           
  inflating: fdda50025.jpg           
  inflating: b642373be.jpg           
  inflating: 38d49de1f.jpg           
  inflating: e0bc48ac2.jpg           
  inflating: 4e2cf9e48.jpg           
  inflating: 5c5e91098.jpg           
  inflating: d6a6539af.jpg           
  inflating: d102de015.jpg           
  inflating: 3308c3a19.jpg           
  inflating: dfbcd5e5b.jpg           
  inflating: b5bf598c6.jpg           
  inflating: 665c7dade.jpg           
  inflating: 1a9116d41.jpg           
  inflating: 2b4a4e5f7.jpg           
  inflating: 00fee3975.jpg           
  inflating: fad7fd94b.jpg           
  inflating: 6c6895185.jpg    

In [0]:
print(f"There are {len(os.listdir('train'))} images in train dataset with {train_df.Id.nunique()} unique classes.")
print(f"There are {len(os.listdir('test'))} images in test dataset.")

FileNotFoundError: ignored